## Exploratory Data Analysis (visualizations)

In [1]:
nums_cols=['credit_score','age','tenure','balance','products_number','estimated_salary']

for cols in nums_cols:
    plt.figure(figsize=(6,4))
    sns.kdeplot(data=df, x=cols, hue='churn', fill=True, alpha=0.5, common_norm=False)  #
    plt.title(f'Distribution of {cols} by Churn Status')
    plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Pair plot 
sns.pairplot(df,vars=['credit_score','age','balance','estimated_salary'],hue='churn',diag_kind='kde',plot_kws={'alpha':0.5})
plt.suptitle('Pairwise Relationship among key Numeric Features',y=1.02)
plt.show()

In [ ]:
# Violin plots 
sns.violinplot(x='churn', y='age', data=df)
plt.title('Age Distribution by Churn Status')
plt.show()

In [ ]:
# Tenure vs Churn
sns.countplot(x='tenure', hue='churn', data=df)
plt.title('Tenure vs Churn Count')
plt.show()

In [ ]:
# Gender vs Churn
sns.countplot(x='gender', hue='churn', data=df)
plt.title('Gender vs Churn Count')
plt.show()